### **INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [3]:
#@ INSTALLING DEPENDENCIES: UNCOMMENT BELOW: 
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [4]:
#@ DOWNLOADING LIBRARIES AND DEPENDENCIES: 
from fastbook import *                              # Getting all the Libraries. 
from fastai.callback.fp16 import *
from fastai.text.all import *                       # Getting all the Libraries.

### **GETTING THE DATA:**
- I will use **Human Numbers** dataset here. It contains the first 10000 numbers written out in English. 

In [6]:
#@ GETTING THE DATASET: 
path = untar_data(URLs.HUMAN_NUMBERS)               # Path to the Dataset. 
path.ls()                                           # Inspecting the Dataset. 

(#2) [Path('/root/.fastai/data/human_numbers/train.txt'),Path('/root/.fastai/data/human_numbers/valid.txt')]

In [7]:
#@ JOINING AND INSPECTING THE DATASET: 
lines = L()                                         # Initializing a List. 
with open(path/"train.txt") as f:                   # Opening the File. 
    lines += L(*f.readlines())                      # Reading the Lines. 
with open(path/"valid.txt") as f:                   # Opening the File. 
    lines += L(*f.readlines())                      # Reading the Lines. 
lines                                               # Inspection. 

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

In [8]:
#@ PREPARING THE DATASET: 
text = " . ".join([l.strip() for l in lines])       # Preparing the Dataset. 
text[:100]                                          # Inspection. 

'one . two . three . four . five . six . seven . eight . nine . ten . eleven . twelve . thirteen . fo'

**Note:**
- I will tokenize the dataset by splitting on spaces. Then I will create a list of unique tokens called vocab for **Numericalization**. Then I will convert the tokens into numbers by looking up in the index of each in the vocab. 

In [9]:
#@ TOKENIZING THE DATASET: 
tokens = text.split(" ")                            # Splitting into Tokens. 
tokens[:10]                                         # Inspecting Tokens. 

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

In [10]:
#@ GETTING UNIQUE TOKENS: 
vocab = L(*tokens).unique()                         # Getting Unique Tokens. 
vocab                                               # Inspection. 

(#30) ['one','.','two','three','four','five','six','seven','eight','nine'...]

In [11]:
#@ CONVERTING TOKENS INTO NUMBERS: 
word2idx = {w:i for i,w in enumerate(vocab)}       # Getting Index of Tokens. 
nums = L(word2idx[i] for i in tokens)              # Converting into Numbers. 
nums                                               # Inspection.  

(#63095) [0,1,2,1,3,1,4,1,5,1...]

### **LANGUAGE MODEL FROM SCRATCH:**
- Here I will create a list of every sequence of three words as independent variables and the next word after each sequence as the dependent variable. 

In [12]:
#@ CREATING SEQUENCE OF TOKENS: 
L((tokens[i:i+3], tokens[i+3]) for i in range(0, len(tokens)-4, 3))

(#21031) [(['one', '.', 'two'], '.'),(['.', 'three', '.'], 'four'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'seven'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'ten'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'thirteen'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'sixteen')...]

In [13]:
#@ CREATING SEQUENCE OF TENSORS FOR NUMERICALIZED VALUES: 
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0, len(nums)-4, 3))   # Creating Sequence.
seqs                                                                           # Inspection.  

(#21031) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 7),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 10),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 13),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 16)...]

In [14]:
#@ CREATING DATALOADERS: 
bs = 64                                                 # Initializing Batchsize. 
cut = int(len(seqs) * 0.8)                              # Initialization. 
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], 
                             bs=bs, shuffle=False)      # Initializing Data Loaders. 

**LANGUAGE MODEL:**
- I will create neural network architecture that takes three words as input and returns the predictions of the probability of each possible next word in the vocab. I will use three standard linear layers. The first linear layer will use only the first words embedding as activations. The second layer will use the second words embedding plus the first layers output activations and the third layer will use the third words embedding plus the second layers output activations. The key effect is that every word is interpreted in the information context of any words preceding it. Each of these three layers will use the same weight matrix. 

In [15]:
#@ LANGUAGE MODEL IN PYTORCH: SIMPLE LINEAR MODEL: 
class LMModel1(Module):                                         # Defining Language Model Class. 
    def __init__(self, vocab_sz, n_hidden):                     # Initializing Constructor Function. 
        self.i_h = nn.Embedding(vocab_sz, n_hidden)             # Initializing Embedding Layer. 
        self.h_h = nn.Linear(n_hidden, n_hidden)                # Initializing Linear Layer. 
        self.h_o = nn.Linear(n_hidden, vocab_sz)                # Initializing Linear Layer. 
    
    def forward(self, x):                                       # Forward Propagation Function. 
        h = F.relu(self.h_h(self.i_h(x[:, 0])))                 # Implementation of RELU. 
        h = h + self.i_h(x[:, 1])                               # Second Word Embeddings and Activations. 
        h = F.relu(self.h_h(h))                                 # Implementation of RELU. 
        h = h + self.i_h(x[:, 2])                               # Third Word Embeddings and Activations. 
        h = F.relu(self.h_h(h))                                 # Implementation of RELU. 
        return self.h_o(h)                                      # Implementation of Linear Layer. 

In [16]:
#@ TRAINING THE LANGUAGE MODEL: 
learn = Learner(dls, LMModel1(len(vocab), 64),                  # Initializing Learner with Language Model. 
                loss_func=F.cross_entropy, metrics=accuracy)    # Initializing Cross Entropy Loss Function. 
learn.fit_one_cycle(4, 1e-3)                                    # Training the Model. 

epoch,train_loss,valid_loss,accuracy,time
0,1.824297,1.970941,0.467554,00:01
1,1.386973,1.823242,0.467554,00:01
2,1.417556,1.654498,0.494414,00:01
3,1.376440,1.650849,0.494414,00:01


In [17]:
#@ MODEL EVALUATION: 
n, counts = 0, torch.zeros(len(vocab))                          # Initialization. 
for x, y in dls.valid: 
    n += y.shape[0]
    for i in range_of(vocab):
        counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)                                      # Common Tensor Index. 
idx, vocab[idx.item()], counts[idx].item()/n                    # Inspection. 

(tensor(29), 'thousand', 0.15165200855716662)